# Cleaning Notebook for Final Dataset
- Merges county and state shapefiles to the datasets
- Adds the pareto efficient column and calculates for each county

In [1]:
# impport necessary libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [23]:
#load data
final = pd.read_csv('/Users/sahana_krishnaraj/Github/siting-optimizer-lmichallange/true_final.csv')
county = gpd.read_file('/Users/sahana_krishnaraj/Github/siting-optimizer-lmichallange/tl_2025_us_county/tl_2025_us_county.shp')
state = gpd.read_file('/Users/sahana_krishnaraj/Github/siting-optimizer-lmichallange/tl_2025_us_state/tl_2025_us_state.shp')

DataSourceError: /Users/sahana_krishnaraj/Github/siting-optimizer-lmichallange/tl_2025_us_county/tl_2025_us_county.shp: No such file or directory

In [ ]:
#view the data
final.head()

,GEOID,Latitude,Longitude,geometry,Gas Production Quantity,Oil Production Quantity,Wind Plant Capacity,Wildfire Hazard Potential Score,Drought Risk Score,County,State,State Name,Pareto Efficient
0,15001,18.978000,-155.688000,POINT (-155.688 18.978),0.000000,0.000000,0.334645,0.117,0.106041,Hawaii County,HI,Hawaii,False
1,15009,20.800100,-156.539000,POINT (-156.539 20.8001),0.000000,0.000000,0.334645,0.100,0.181957,Maui County,HI,Hawaii,False
2,15003,21.669200,-157.950100,POINT (-157.9501 21.6692),0.000000,0.000000,0.371260,0.197,0.044702,Honolulu County,HI,Hawaii,False
3,15003,21.680400,-157.982000,POINT (-157.982 21.6804),0.000000,0.000000,0.382505,0.197,0.044702,Honolulu County,HI,Hawaii,False
4,48061,26.118900,-97.343100,POINT (-97.3431 26.1189),0.000000,0.000000,0.540691,0.103,0.000000,Cameron County,TX,Texas,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,2185,70.169433,-146.316676,POINT (-146.3166757 70.16943313),0.674542,0.000000,0.000000,0.151,0.100000,North Slope Borough,AK,Alaska,False
1191,2185,70.309122,-148.670664,POINT (-148.6706638 70.30912164),0.000000,0.955906,0.000000,0.151,0.100000,North Slope Borough,AK,Alaska,False
1192,2185,70.324819,-149.945492,POINT (-149.9454917 70.32481918),0.000000,0.772550,0.000000,0.151,0.100000,North Slope Borough,AK,Alaska,False
1193,2185,70.331718,-147.879900,POINT (-147.8799 70.33171813),0.764759,0.000000,0.000000,0.151,0.100000,North Slope Borough,AK,Alaska,False


In [24]:
#merge the data dataframes - cn geometry refers to the polygon
countstate = pd.merge(county, state[["STATEFP","STUSPS","NAME"]], on="STATEFP", how="left")
countstate.rename(columns={"geometry": "cn geometry"}, inplace=True)
countstate.head()

NameError: name 'county' is not defined

In [ ]:
#fill values that have no leading 0 because GEOID should be 5 digits
final['GEOID'] = final['GEOID'].astype(str)
final["GEOIDlen"] = final['GEOID'].str.len()
final["GEOIDlen"].value_counts()
final['GEOID'] = final['GEOID'].str.zfill(5)
final = final.drop(columns=['GEOIDlen'])
final['GEOID'] = final['GEOID'].astype('int64')

In [ ]:
#sanity check for countstate GEOID formatting
countstate['GEOID'] = countstate['GEOID'].astype(str)
countstate['GEOIDlen'] = countstate['GEOID'].str.len()
countstate = countstate.drop(columns=['GEOIDlen'])
countstate['GEOID'] = countstate['GEOID'].astype('int64')

In [ ]:
#merge final and countstate
merged = pd.merge(final, countstate[["NAMELSAD","STUSPS","NAME_y","GEOID"]], on="GEOID", how="left")
merged = merged.drop(columns=["NAMELSAD", "STUSPS","NAME_y"], inplace=True)

NameError: name 'countstate' is not defined

In [ ]:
"""# For production columns: NA = 0 (no production = no resource)
merged['Wind Plant Capacity'] = merged['Wind Plant Capacity'].fillna(0)
merged['Oil Production Quantity'] = merged['Oil Production Quantity'].fillna(0)
merged['Gas Production Quantity'] = merged['Gas Production Quantity'].fillna(0)

# For risk columns: NA = median (unknown risk = average risk)
merged['Wildfire Hazard Potential Score'] = merged['Wildfire Hazard Potential Score'].fillna(merged['Wildfire Hazard Potential Score'].median())
merged['Drought Risk Score'] = merged['Drought Risk Score'].fillna(merged['Drought Risk Score'].median())
merged.columns.values[11] = "State Name" 
"""


In [ ]:

#there were duplicate entries so this controls for them
merged = merged.groupby('GEOID', as_index=False).agg({
    'County': 'first',
    'State': 'first',
    'State Name': 'first',
    'Wildfire Hazard Potential Score': 'mean',
    'Drought Risk Score': 'mean',
    'Wind Plant Capacity': 'sum',  # Total capacity across all plants
    'Oil Production Quantity': 'sum',  # Total production
    'Gas Production Quantity': 'sum',
    'Latitude': 'first',
    'Longitude': 'first',
    'geometry': 'first',  # Keep the geometry for mapping
    'Pareto Efficient': 'first',
    'cn geometry': 'first'
})


# Compute Pareto Efficiency for each county 
- True if county is Pareto Efficient
- False if county is not

In [ ]:
file = pd.read_csv("true_final.csv")
file["Pareto Efficient"] = True

In [5]:
file.head()

,GEOID,County,State,State Name,Wildfire Hazard Potential Score,Drought Risk Score,Wind Plant Capacity,Oil Production Quantity,Gas Production Quantity,Latitude,Longitude,geometry,Pareto Efficient,cn geometry
0,2020,Anchorage Municipality,AK,Alaska,0.151,0.1,0.314185,0.000000,0.000000,61.139500,-150.216000,POINT (-150.216 61.1395),True,"POLYGON ((-150.409942 61.168174, -150.235184 6..."
1,2122,Kenai Peninsula Borough,AK,Alaska,0.151,0.1,0.000000,0.000000,1.759298,59.850499,-151.567667,POINT (-151.5676667 59.85049932),True,"POLYGON ((-150.572778 59.282299, -150.591243 5..."
2,2150,Kodiak Island Borough,AK,Alaska,0.151,0.1,0.141177,0.000000,0.000000,57.786900,-152.440600,POINT (-152.4406 57.7869),True,"MULTIPOLYGON (((-152.466527 58.914995, -152.46..."
3,2185,North Slope Borough,AK,Alaska,0.151,0.1,0.000000,2.291418,1.439301,70.169433,-146.316676,POINT (-146.3166757 70.16943313),True,"POLYGON ((-166.971742 68.345199, -166.971567 6..."
4,4003,Cochise County,AZ,Arizona,0.151,0.1,0.382505,0.000000,0.000000,32.309600,-110.087000,POINT (-110.087 32.3096),True,"POLYGON ((-110.452315 31.760209, -110.452314 3..."


In [6]:

minimize = ["Wildfire Hazard Potential Score", "Drought Risk Score", "Oil Production Quantity", "Gas Production Quantity"] 
maximize = ["Wind Plant Capacity"]
for i in range(len(file)):
    for j in range(len(file)): #nested loop to comare within 
        if i == j:
            continue #skip the same row comparison -- redundant
        # check if row j is better than row i
        #all() function returns true if all elements in the iterable are true

        #the alls ensure that for a row to dominate another row, it has to be better in all the minimize and maximize columns
        all_min = all(file.iloc[j][minimize] <= file.iloc[i][minimize]) #checks each column in current row and compares it with the column in the previous row (dominates if they are lower than previous good!)
        all_max = all(file.iloc[j][maximize] >= file.iloc[i][maximize]) #checks each column in current row and compares it with the column in the previous row (dominates if they are higher than previous good!)
        
        #the anys ensure that for a row to dominate another row, it has to be strictly better in at least one of the minimize or maximize columns (like an edge case for when two rows have same values but how do u see if its better)
        any_strict = any(file.iloc[j][minimize] < file.iloc[i][minimize]) or \
            any(file.iloc[j][maximize] > file.iloc[i][maximize])

        #finally check if all conditions are met
        if all_min and all_max and any_strict:
            file.iloc[i, file.columns.get_loc("Pareto Efficient")] = False
            break #no need to check further, move to next row

In [7]:
#check the value counts of pareto efficient
file["Pareto Efficient"].value_counts()

Pareto Efficient
False    484
True      10
Name: count, dtype: int64

In [9]:
file.to_csv("changed_data.csv",index=False)

In [ ]:
merged.to_csv("dataset.csv", index=False)